This was an attempt to integrate snnTorch into a ResNet written with pytorch. However, 

In [ ]:
from DataLoader import MyOwnDataloader
from pycocotools.coco import COCO

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import torchvision

import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

from functools import partial
from dataclasses import dataclass
from collections import OrderedDict

In [ ]:
torch.cuda.empty_cache() 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved

from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total/1000000}')
print(f'free     : {info.free/1000000}')
print(f'used     : {info.used/1000000}')

In [ ]:
dtype = torch.float

In [ ]:
dtype = torch.float
dataDir='/media/gamedisk/COCO_dataset/'
val='val2017'
train = 'train2017'
test = 'test2017'

val_annFile='{}/annotations/instances_{}.json'.format(dataDir,val)
train_annFile='{}/annotations/instances_{}.json'.format(dataDir,train)
test_annFile='/media/gamedisk/COCO_dataset/annotations/image_info_test2017.json' 
# Batch size
batch_size = 32


classes = {
    "bird": 1,
    "cat": 2,
    "dog": 3,
    "horse": 4,
    "sheep": 5,
    "cow": 6,
    "elephant": 7,
    "bear": 8,
    "zebra": 9,
    "giraffe": 10
}


train_loader = MyOwnDataloader(dataDir = dataDir, dataType = train,
                     annFile = train_annFile, classes = classes, train_batch_size=batch_size)
train_dl = train_loader.concat_datasets()


# test_loader = MyOwnDataloader(dataDir = dataDir, dataType = test,
#                      annFile = test_annFile, classes = classes, train_batch_size=batch_size)
# test_dl = test_loader.concat_datasets()


val_loader = MyOwnDataloader(dataDir = dataDir, dataType = val,
                     annFile = val_annFile, classes = classes, train_batch_size=batch_size)
valid_dl = val_loader.concat_datasets()

In [ ]:
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
num_steps = 50

In [ ]:
class Network(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, timesteps: int):
        super(Network, self).__init__()
        self.timesteps = timesteps
        # CNNs for rgb images
        self.conv1 = nn.Conv2d(in_channels, 32,
                        kernel_size=7,
                        padding=3,
                        # no bias because it is not bio-plausible (and hard to impl in neuromorphic hardware)
                        bias=True,
                        dilation=1,
                        stride=2)
        self.lif1 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.conv2 = nn.Conv2d(32, 64,
                               kernel_size=3,
                               padding=1,
                               # no bias because it is not bio-plausible (and hard to impl in neuromorphic hardware)
                               bias=True,
                               stride=2)
        self.lif2 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.conv3 = nn.Conv2d(64, 128,
                        kernel_size=3,
                        padding=1,
                        bias=True,
                        stride=2)
        self.lif3 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.conv4 = nn.Conv2d(128, 256,
                        kernel_size=3,
                        padding=1,
                        # no bias because it is not bio-plausible (and hard to impl in neuromorphic hardware)
                        bias=True,
                        stride=2)
        self.lif4 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.conv5 = nn.Conv2d(256, 512,
                    kernel_size=3,
                    padding=1,
                    bias=True,
                    stride=2)
        self.lif5 = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(slope=25), init_hidden=False)

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))


        self.flat = nn.Flatten()
        self.dropout = nn.Dropout()
        self.fc = nn.Linear(512, out_channels, bias=True)
        self.fc_spike = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(
            slope=25), init_hidden=False, output=True)


        # # Connecting CNN outputs with Fully Connected layers for classification

        self.class_fc1 = nn.Linear(in_features=192*3*3, out_features=240)
        self.class_fc2 = nn.Linear(in_features=240, out_features=120)
        self.class_out = nn.Linear(in_features=120, out_features=10)

        self.fc_spike = snn.Leaky(beta=0.5, spike_grad=surrogate.fast_sigmoid(
            slope=25), init_hidden=False, output=True)

        Connecting CNN outputs with Fully Connected layers for bounding box
        self.box_fc1 = nn.Linear(in_features=192*3*3, out_features=240)
        self.box_fc2 = nn.Linear(in_features=240, out_features=120)
        self.box_out = nn.Linear(in_features=120, out_features=4)


    def forward(self, inputs):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()
        mem3 = self.lif3.init_leaky()
        mem4 = self.lif4.init_leaky()
        mem5 = self.lif5.init_leaky()

        mem_fc_spike = self.fc_spike.init_leaky()

        # Record the final layer
        spk5_rec = []
        mem5_rec = []

        for k in range(self.timesteps):
            t = inputs[k, :, :, :]
            # t = inputs
            t = self.conv1(t)
            t = F.max_pool2d(t, kernel_size=2)
            t, mem1 = self.lif1(t, mem1)

            t = self.conv2(t)
            t = F.max_pool2d(t, kernel_size=2,)
            t, mem2 = self.lif2(t, mem2)


            t = self.conv3(t)
            t = F.max_pool2d(t, kernel_size=2)
            t, mem3 = self.lif3(t, mem3)


            t = self.conv4(t)
            t = F.max_pool2d(t, kernel_size=2)
            t, mem4 = self.lif4(t, mem4)


            t = self.conv5(t)
            class_t, mem5 = self.lif5(t, mem5)
            # t = F.relu(t)
            # t = F.avg_pool2d(t, 1)

            class_t = self.avg_pool(class_t)
            # t = torch.flatten(t,start_dim=1)

            class_t = self.flat(class_t)
            class_t = self.dropout(class_t)
            class_t = class_t.T
            class_t = self.fc(class_t)
            class_t, mem_fc_spike = self.fc_spike(class_t, mem_fc_spike)
            spk5_rec.append(class_t)
            mem5_rec.append(mem_fc_spike)

            # classifier
            # class_t = self.class_fc1(t)
            # class_t = F.relu(class_t)

            # class_t = self.class_fc2(class_t)
            # class_t = F.relu(class_t)


            # class_t, mem_fc_spike = self.fc_spike(class_t, mem_fc_spike)



            # box_t = self.box_fc1(t)
            # box_t = F.relu(box_t)

            # box_t = self.box_fc2(box_t)
            # box_t = F.relu(box_t)

            # box_t = self.box_out(box_t)
            # box_t = F.sigmoid(box_t)

        return torch.stack(spk5_rec, dim=0), torch.stack(mem5_rec, dim=0)

In [ ]:
model = Network(3, 10, 8)
model = model.to(device)

In [ ]:
def forward_pass(net, num_steps, data):
  mem_rec = []
  spk_rec = []
  utils.reset(net)  # resets hidden states for all LIF neurons in net

  for step in range(num_steps):
      spk_out, mem_out = net(data)
      spk_rec.append(spk_out)
      mem_rec.append(mem_out)

  return torch.stack(spk_rec), torch.stack(mem_rec)

In [ ]:
def batch_accuracy(train_loader, net, num_steps):
  with torch.no_grad():
    total = 0
    acc = 0
    net.eval()

    # train_loader = iter(train_loader)
    for batch, (images, annotations) in tqdm(enumerate(train_loader)):
      imgs = list(img.to(device) for img in images)
      neural_images = torch.stack(imgs)
      x = neural_images.to(device)
      annotations = [{k: v for k, v in t.items()} for t in annotations]
      y = annotations[0]['labels'].to(device)
      z = annotations[0]['boxes'].to(device)
      print(len(y))
      try:
        spk_rec, _ = forward_pass(net, num_steps, x)

        acc += SF.accuracy_rate(spk_rec[0], y[0]) * spk_rec.size(1)
        total += spk_rec.size(1)
      except:
        pass

  return acc/total

In [ ]:
torch.cuda.empty_cache()
test_acc = batch_accuracy(valid_dl, model, num_steps)

In [ ]:
print(f"The total accuracy on the test set is: {test_acc * 100:.2f}%")

In [ ]:
spk_rec, mem_rec = forward_pass(model, num_steps, data)

In [ ]:
def print_batch_accuracy(data, targets, train=False):
    output, _ = model(data.view(batch_size, -1))
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

def train_printer():
    print(f"Epoch {epoch}, Iteration {iter_counter}")
    print(f"Train Set Loss: {loss_hist[counter]:.2f}")
    print(f"Test Set Loss: {test_loss_hist[counter]:.2f}")
    print_batch_accuracy(data, targets, train=True)
    print_batch_accuracy(test_data, test_targets, train=False)
    print("\n")

In [ ]:
loss = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, betas=(0.9, 0.999))


for batch, (images, annotations) in tqdm(enumerate(train_dl)):
  imgs = list(img.to(device) for img in images)
  neural_images = torch.stack(imgs)
  x = neural_images.to(device)
  annotations = [{k: v for k, v in t.items()} for t in annotations]
  y = annotations[0]['labels'].to(device)
  z = annotations[0]['boxes'].to(device)
  data = x
  targets = y
  print(targets)
  if batch == 1:
    break

# data = data.to(device)
# targets = targets.to(device)
print(data.shape)

spk_rec, mem_rec = model(data)
print(mem_rec.size())

# initialize the total loss value
loss_val = torch.zeros((1), dtype=dtype, device=device)

# sum loss at every step
for step in range(num_steps):
  loss_val += loss(mem_rec[step], targets)

print(f"Training loss: {loss_val.item():.3f}")

print_batch_accuracy(data, targets, train=True)

In [ ]:
loss_fn = SF.ce_rate_loss()

In [ ]:
def batch_accuracy(train_loader, net, num_steps):
  with torch.no_grad():
    total = 0
    acc = 0
    net.eval()

    train_loader = iter(train_loader)
    for data, targets in train_loader:
      data = data.to(device)
      targets = targets.to(device)
      spk_rec, _ = forward_pass(net, num_steps, data)

      acc += SF.accuracy_rate(spk_rec, targets) * spk_rec.size(1)
      total += spk_rec.size(1)

  return acc/total

In [ ]:
def train(model):
    # Defining the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, betas=(0.9, 0.999))
    num_of_epochs = 10
    epochs = []
    losses = []
    test_acc_hist = []
    get_num_correct = []
    # Creating a directory for storing models

    for epoch in range(num_of_epochs):
        tot_loss = 0
        tot_correct = 0
        train_start = time.time()
        model.train()


        for batch, (images, annotations) in tqdm(enumerate(train_dl)):

        	# Converting data from cpu to GPU if available to improve speed
            imgs = list(img.to(device) for img in images)
            neural_images = torch.stack(imgs)
            x = neural_images.to(device)
            annotations = [{k: v for k, v in t.items()} for t in annotations]
            y = annotations[0]['labels'].to(device)
            z = annotations[0]['boxes'].to(device)
            avg_loss = backprop.BPTT(model, train_loader, optimizer=optimizer, criterion=loss_fn,
                            num_steps=num_steps, time_var=False, device=device)
            # Sets the gradients of all optimized tensors to zero
            optimizer.zero_grad()
            [y_pred,z_pred]= model(x)
            # Compute loss (here CrossEntropyLoss)
            class_loss = F.cross_entropy(y_pred, y)
            box_loss = F.mse_loss(z_pred, z)
            (box_loss + class_loss).backward()
            # class_loss.backward()
            optimizer.step()
            print("Train batch:", batch+1, " epoch: ", epoch, " ",
                  (time.time()-train_start)/60, end='\r')

        model.eval()
        for batch, (images, annotations) in tqdm(enumerate(valid_dl)):
        	# Converting data from cpu to GPU if available to improve speed	
            imgs = list(img.to(device) for img in images)
            neural_images = torch.stack(imgs)
            x = neural_images.to(device)
            annotations = [{k: v for k, v in t.items()} for t in annotations]
            y = annotations[0]['labels'].to(device)
            z = annotations[0]['boxes'].to(device)
            # Sets the gradients of all optimized tensors to zero
            optimizer.zero_grad()
            with torch.no_grad():
                [y_pred,z_pred]= model(x)
                
                # Compute loss (here CrossEntropyLoss)
                class_loss = F.cross_entropy(y_pred, y)
                box_loss = F.mse_loss(z_pred, z)
                # Compute loss (here CrossEntropyLoss)

            tot_loss += (class_loss.item() + box_loss.item())
            tot_correct += get_num_correct(y_pred, y)
            print("Test batch:", batch+1, " epoch: ", epoch, " ",
                  (time.time()-train_start)/60, end='\r')
        epochs.append(epoch)
        losses.append(tot_loss)
        print("Epoch", epoch, "Accuracy", (tot_correct)/2.4, "loss:",
              tot_loss, " time: ", (time.time()-train_start)/60, " mins")
        torch.save(model.state_dict(), "model_ep"+str(epoch+1)+".pth")

In [ ]:
train(model)